In [1]:
sc

In [2]:
sc.version

u'2.0.2'

The Data Science Experience notebook environment predefines the Spark context.
In other environments, you need to pick an interpreter (for example, pyspark for Python) and create a SparkConf object to initialize a SparkContext object. 

**For example: **

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName(appName).setMaster(master)

sc = SparkContext(conf=conf)

In [3]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # a collection

In [4]:
x_nbr_rdd = sc.parallelize(x) # RDD

In [5]:
x_nbr_rdd.first()

1

In [6]:
x_nbr_rdd.take(5)

[1, 2, 3, 4, 5]

In [8]:
y = ["Hi everyone", "My Name is Kate"]
y_str_rdd = sc.parallelize(y)
y_str_rdd.take(1)

['Hi everyone']

In [9]:
x_nbr_rdd_2 = x_nbr_rdd.map(lambda x: x+1)
x_nbr_rdd_2.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [10]:
X = ["1,2,3,4,5,6,7,8,9,10"] # we neeed "" to create an array, not a collection

In [11]:
y_rd = sc.parallelize(X)
Sum_rd = y_rd.map(lambda y: y.split(",")).\
map(lambda y: (int(y[2])+int(y[9])))
Sum_rd.first()

13

In [12]:
Words = ["Hello Human. I'm Apache Spark and I love running analysis on data."]
words_rd = sc.parallelize(Words)
words_rd.first()

"Hello Human. I'm Apache Spark and I love running analysis on data."

In [13]:
Words_rd2 = words_rd.map(lambda line: line.split(" "))
Words_rd2.first()

['Hello',
 'Human.',
 "I'm",
 'Apache',
 'Spark',
 'and',
 'I',
 'love',
 'running',
 'analysis',
 'on',
 'data.']

In [14]:
Words_rd2.count()

1

In [15]:
words_rd2 = words_rd.flatMap(lambda line: line.split(" "))
words_rd2.take(3)

['Hello', 'Human.', "I'm"]

In [16]:
z = ["First,Line", "Second,Line", "and,Third,Line"]
z_str_rdd = sc.parallelize(z)
z_str_rdd.first()

'First,Line'

In [17]:
z_str_rdd_split_flatmap = z_str_rdd.flatMap(lambda line: line.split(","))
z_str_rdd_split_flatmap.collect()

['First', 'Line', 'Second', 'Line', 'and', 'Third', 'Line']

### "Map-Reduce" in Spark

In [19]:
countWords = z_str_rdd_split_flatmap.map(lambda word:(word,1))
countWords.collect()

[('First', 1),
 ('Line', 1),
 ('Second', 1),
 ('Line', 1),
 ('and', 1),
 ('Third', 1),
 ('Line', 1)]

In [20]:
from operator import add
countWords2 = countWords.reduceByKey(add)
countWords2.collect()

[('and', 1), ('Line', 3), ('Second', 1), ('Third', 1), ('First', 1)]

###  Filter

In [21]:
words_rd3 = z_str_rdd_split_flatmap.filter(lambda line: "Second" in line) 

print "The count of words " + str(words_rd3.first())
print "Is: " + str(words_rd3.count())

The count of words Second
Is: 1


### Get the file from a URL

In [22]:
!rm README.md* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md

--2017-09-21 14:42:50--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3624 (3.5K) [text/plain]
Saving to: ‘README.md’

100%[======================================>] 3,624       --.-K/s   in 0s      

2017-09-21 14:42:50 (20.1 MB/s) - ‘README.md’ saved [3624/3624]



In [23]:
textfile_rdd = sc.textFile("README.md")
textfile_rdd.count()

98

In [24]:
textfile_rdd.take(5)

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, and Python, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a']

In [25]:
Spark_lines = textfile_rdd.filter(lambda line: "Spark" in line)
Spark_lines.first()

u'# Apache Spark'

In [26]:
print "The file README.md has " + str(Spark_lines.count()) + \
" of " + str(textfile_rdd.count()) + \
" Lines with word Spark in it."

The file README.md has 19 of 98 Lines with word Spark in it.


In [27]:
temp = Spark_lines.flatMap(lambda line:line.split(" ")).map(lambda x:(x,1)).reduceByKey(add)
temp.filter(lambda (k,v): k.startswith("Spark")).collect()

[(u'Spark](#building-spark).', 1),
 (u'Spark', 14),
 (u'SparkPi', 2),
 (u'Spark.', 1),
 (u'Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1)]

In [28]:
temp.filter(lambda (k,v): "Spark" in k).collect()

[(u'Spark](#building-spark).', 1),
 (u'Spark', 14),
 (u'tests](https://cwiki.apache.org/confluence/display/SPARK/Contributing+to+Spark#ContributingtoSpark-AutomatedTesting).',
  1),
 (u'SparkPi', 2),
 (u'Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 (u'Spark.', 1)]

In [29]:
!rm Scores.txt* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
 
Raw_Rdd = sc.textFile("Scores.txt")

SumScores = Raw_Rdd.map(lambda l: l.split(",")).\
map(lambda v : (v[0], int(v[1])+int(v[2])+int(v[3])+int(v[4])))

Final = SumScores.map(lambda avg: (avg[0],avg[1],avg[1]/4.0))

Final.take(5)

--2017-09-21 14:45:31--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75 [text/plain]
Saving to: ‘Scores.txt’

100%[======================================>] 75          --.-K/s   in 0s      

2017-09-21 14:45:31 (13.2 MB/s) - ‘Scores.txt’ saved [75/75]



[(u'Carlo', 15, 3.75),
 (u'Mokhtar', 15, 3.75),
 (u'Jacques', 15, 3.75),
 (u'Braden', 15, 3.75),
 (u'Chris', 15, 3.75)]